In [38]:
import requests
import pandas as pd
import altair as alt

In [3]:
url = "https://elearningindustry.com/content/directory/listings?category=7113&sort=rating&filters[7107_deployment][]=158241&filters[7107_market][]=158325&filters[7107_market][]=158340&filters[7107_market][]=158304&offset=0&limit=1000"
res = requests.get(url)

In [197]:
def process_response(url_json: list) -> dict:
    """ Process HTTP Response
    Flatten nested HTTP request into a simpler document.

    Parameters
    ----------
    url_json: list
        HTTP response json produced using the .json() method from a Response object from the requests library. 
    """
    out = {}
    for dict in url_json:
        out[dict['title']] = {}
        out[dict['title']]['name'] = dict['title']
        out[dict['title']]['url'] = dict['website']
        out[dict['title']]['rating'] = dict['rating']
        out[dict['title']]['review'] = dict['reviewCount']
        out[dict['title']]['rank'] = dict['rank']

        for deployment in dict['deployment']:
            out[dict['title']]['deployment - {dep}'.format(dep = deployment['name'])] = True
        for integration in dict['integrations']:
            out[dict['title']]['integrations - {int}'.format(int = integration['name'])] = True
        for feature in dict['features']:
            out[dict['title']]['features - {cat} - {feat}'.format(cat = feature['parent'], feat = feature['name'])] = True
    return(out)

def process_dataframe(df: pd.DataFrame, only_true: bool = True) -> pd.DataFrame:
    """ Process pd.DataFrame from Response

    Ingest a messy DataFrame and clean it up!
    """
    for column in df.columns:
        if ' - ' in column:
            df[column] = df[column].fillna(False)
    wanted_columns = [col for col in df.columns if ' - ' in col]
    df = df.melt(id_vars=list(set(df.columns) - set(wanted_columns)),
                 value_vars=wanted_columns,
                 var_name='Quality', value_name='Bool')
    df['Type'] = df['Quality'].str.split(' - ').str[0]
    df['Quality'] = df['Quality'].str.split(' - ').str[-1]

    if only_true:
        df = df[df['Bool'] == True]

    return(df)

df = process_dataframe(
        pd.DataFrame.from_dict(
            process_response(res.json()),
            orient='index')
            )
def get_features_axis_order(url_json: list) -> dict:
    out = {}
    for dict in url_json:
        for feature in dict['features']:
            print(feature['parent'])
            if feature['parent'] in out.keys():
                if feature['name'] not in out[feature['parent']]:
                    out[feature['parent']].append(feature['name'])
                else:
                    pass
            else:
                out[feature['parent']] = []
                out[feature['parent']].append(feature['name'])
    for key, value in out.items():
        out[key] = sorted(value)

    return(out)

order_lists = get_features_axis_order(res.json())

158483
158318
159101
159377
159377
158318
158732
158624
158237
158237
158552
158552
158942
159308
158879
158879
158237
159101
158318
158624
158318
159101
158624
158942
159005
159005
159044
158624
158624
158624
158765
158834
158732
159308
158483
158318
158879
158438
158624
159005
158942
159308
158942
158765
158834
158834
158834
158942
158552
159377
159101
158879
158765
158624
158879
158765
159308
159101
158732
159044
158483
158552
158834
159101
159200
159101
158834
158483
158282
158282
159200
158879
159044
158552
159101
159377
158879
158438
158624
158552
158552
158483
158483
159242
158765
159005
159377
159275
158624
158552
158438
158438
158624
158765
159044
159044
158624
158318
158765
158765
158318
159101
158732
158237
158237
158237
158318
159242
158318
158942
158624
158765
158834
158732
159308
158318
158879
158942
159308
158765
158942
159101
158624
158765
159308
158732
158483
159101
159101
158282
159200
158879
158732
158552
159101
158879
158552
158483
158765
159005
158765
158624
158318

In [198]:
order_lists

{158483: ['Active Directory/LDAP Integration',
  'Custom User login page',
  'Manual Accounts',
  'No login',
  'SAML2/API Integration',
  'Self-Registration',
  'Self-Registration w. Admin Confirmation'],
 158318: ['Add a new user',
  'Archive users',
  'Browse list of users',
  'Bulk User Actions',
  'Custom/Mandatory User profile fields',
  'Upload users'],
 159101: ['Additional external pages',
  'Block Management',
  'Calendar Settings',
  'Language settings',
  'Learning Accessibility',
  'Location Settings',
  'Media embedding settings',
  'Multilanguage Support',
  'Ready-made Themes'],
 159377: ['Anti-spam',
  'Anti-virus',
  'IP Blocker',
  'Restrict registration to specific domains',
  'Strong Passwords'],
 158732: ['Assign Courses to categories',
  'Create new Categories',
  'Manage Categories',
  'Priced Categories (Bundle)'],
 158624: ['Assignments Engine',
  'Built-In Authoring Tool',
  'Can reuse PPTs, PDFs, Videos',
  'Changing Course default settings',
  'Consume onli

In [148]:
alt.data_transformers.disable_max_rows()
top20 = df.groupby('name')['review'].max().sort_values(ascending=False)[0:19]
alt.Chart(data = df[(df['Type'] == 'features') & (df['name'].isin(top20.index))],
          title="Top 20 SaaS LMS for Academic, Non Profit, or Small Businesses \n"
          ).mark_rect().encode(
    alt.X('Quality').title(""),
    alt.Y('name', sort=list(top20.index)).title(""),
    alt.Color('Bool', legend=None)
).configure_view(
    step=13,
    strokeWidth=0
).configure_title(
    anchor='start'
)

alt.Chart(...)

Moodle                                Moodle
Xperiencify                      Xperiencify
Tovuti LMS                        Tovuti LMS
Cornerstone Learning    Cornerstone Learning
Chamilo                              Chamilo
                                ...         
Yeira                                  Yeira
Zaamna LMS                        Zaamna LMS
Zalvadora LMS LXP          Zalvadora LMS LXP
Zoho Learn                        Zoho Learn
Zoho People - LMS          Zoho People - LMS
Name: name, Length: 220, dtype: object